In [16]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import re
import random
from PIL import Image, ImageTk

# Sample Images per category (replace with actual images)
image_paths = {
    "traditional": ["gelede_headdress.jpg", "benin_head.jpg"],
    "modern": ["iwin.png", "dancer.jpg"],
    "contemporary": ["gelede_celebration.jpg", "common_goal.jpg"]
}

def is_valid_email(email):
    return re.match(r"[^@]+@[^@]+\.[^@]+", email)

def apply_transformation(image, transformation):
    if transformation == "Grayscale":
        return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    elif transformation == "Edge Detection":
        return cv2.Canny(image, 100, 200)
    elif transformation == "Sharpen":
        kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
        return cv2.filter2D(image, -1, kernel)
    return image

def resize_image(image, width, height):
    return image.resize((width, height), Image.Resampling.LANCZOS)

def process_input():
    email = email_var.get()
    age = int(age_var.get())
    category = category_var.get()
    transformation = transform_var.get()
    
    if not is_valid_email(email):
        messagebox.showerror("Error", "Invalid email. Please enter a valid email.")
        return
    if age < 18:
        messagebox.showerror("Error", "You must be above 18 to access the collection.")
        return
    if category not in image_paths:
        messagebox.showerror("Error", "Invalid category selection.")
        return
    
    selected_images = random.sample(image_paths[category], 2)  # Select two images
    images = [cv2.imread(img) for img in selected_images]
    if any(img is None for img in images):
        messagebox.showerror("Error", "Image not found. Replace with actual image paths.")
        return
    
    transformed_images = [apply_transformation(img, transformation) for img in images]
    
    # Convert images for Tkinter
    images = [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in images]
    images = [Image.fromarray(img) for img in images]
    transformed_images = [Image.fromarray(img) for img in transformed_images]
    
    # Resize images to fit within the window
    img_width, img_height = 300, 300
    images = [resize_image(img, img_width, img_height) for img in images]
    transformed_images = [resize_image(img, img_width, img_height) for img in transformed_images]
    
    img1, img2 = ImageTk.PhotoImage(images[0]), ImageTk.PhotoImage(images[1])
    t_img1, t_img2 = ImageTk.PhotoImage(transformed_images[0]), ImageTk.PhotoImage(transformed_images[1])
    
    original_label1.config(image=img1)
    original_label1.image = img1
    original_label2.config(image=img2)
    original_label2.image = img2
    
    transformed_label1.config(image=t_img1)
    transformed_label1.image = t_img1
    transformed_label2.config(image=t_img2)
    transformed_label2.image = t_img2

# GUI Setup
root = tk.Tk()
root.title("Art Image Transformer")
root.geometry("800x600")

# Input Fields
email_var = tk.StringVar()
tk.Label(root, text="Email:").grid(row=0, column=0)
tk.Entry(root, textvariable=email_var).grid(row=0, column=1)

age_var = tk.StringVar()
tk.Label(root, text="Age:").grid(row=1, column=0)
tk.Entry(root, textvariable=age_var).grid(row=1, column=1)

category_var = tk.StringVar()
tk.Label(root, text="Category:").grid(row=2, column=0)
ttk.Combobox(root, textvariable=category_var, values=["traditional", "modern", "contemporary"]).grid(row=2, column=1)

transform_var = tk.StringVar()
tk.Label(root, text="Transformation:").grid(row=3, column=0)
ttk.Combobox(root, textvariable=transform_var, values=["Grayscale", "Edge Detection", "Sharpen"]).grid(row=3, column=1)

# Submit Button
tk.Button(root, text="Submit", command=process_input).grid(row=4, column=0, columnspan=2)

# Image Display
original_label1 = tk.Label(root)
original_label1.grid(row=5, column=0, padx=10, pady=10)
original_label2 = tk.Label(root)
original_label2.grid(row=5, column=1, padx=10, pady=10)
transformed_label1 = tk.Label(root)
transformed_label1.grid(row=6, column=0, padx=10, pady=10)
transformed_label2 = tk.Label(root)
transformed_label2.grid(row=6, column=1, padx=10, pady=10)

root.mainloop()
